In [28]:
import pandas as pd
import numpy as np
import re
import plotly as iplot
import cufflinks as cf

import seaborn as sns

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_csv(r"F:\FYP-II-master\data\training.csv")
data.head()

,class,text,Unnamed: 2
0,PI,What's the difference between the iPhone X a...,NaN
1,no,@ siracusa just listened to Hypercritical #16...,NaN
2,PI,Same. IPhone X won't be bad https:// twitt...,NaN
3,no,Ive had the iphone 8 plus and iphone x and i...,NaN
4,no,“ iphone x for £150” do i have dickhead writ...,NaN


## Exploratory data analysis

In [3]:
data.shape
data.isnull().sum()

class            0
text             0
Unnamed: 2    1682
dtype: int64

## Data preprocessing

In [4]:
data=data.drop(columns=['Unnamed: 2'])

In [5]:
data['class']=data['class'].replace(['Undefined','undefined'],'un')
data=data[data['class']!='un']
data.head(10)

,class,text
0,PI,What's the difference between the iPhone X a...
1,no,@ siracusa just listened to Hypercritical #16...
2,PI,Same. IPhone X won't be bad https:// twitt...
3,no,Ive had the iphone 8 plus and iphone x and i...
4,no,“ iphone x for £150” do i have dickhead writ...
5,PI,This iPhone X still very sexy a year later
7,PI,If not I'm using my kidney. Omo what is ferrar...
8,PI,iPhone X 's max for sale
9,no,Someone once told me that.....\n\nBut I've nev...
10,PI,screenshots from the iphone x look so clapped


In [6]:
data['class']=data['class'].replace(['no','No'],0)
data['class']=data['class'].replace(['Pl','PI','yes','Yes'],1)
data.head(10)

,class,text
0,1,What's the difference between the iPhone X a...
1,0,@ siracusa just listened to Hypercritical #16...
2,1,Same. IPhone X won't be bad https:// twitt...
3,0,Ive had the iphone 8 plus and iphone x and i...
4,0,“ iphone x for £150” do i have dickhead writ...
5,1,This iPhone X still very sexy a year later
7,1,If not I'm using my kidney. Omo what is ferrar...
8,1,iPhone X 's max for sale
9,0,Someone once told me that.....\n\nBut I've nev...
10,1,screenshots from the iphone x look so clapped


In [7]:
data.shape

(904, 2)

## Text Preprocessing

In [8]:
replace_space=re.compile('[/(){}\[\]\|@,;]')
replace_bad_symbol=re.compile('[^0-9a-z #+_]')
Stopwords=set(stopwords.words('english'))

In [9]:
def clean_text(text):
    text=text.lower()
    text=replace_space.sub(' ',text)
    text=replace_bad_symbol.sub('',text)

    text=' '.join(word for word in text.split() if word not in Stopwords)
    return text

data['text']=data['text'].apply(clean_text)
data.head(10)

,class,text
0,1,whats difference iphone x iphone x x max
1,0,siracusa listened hypercritical #16 whole epis...
2,1,iphone x wont bad https twittercom afuaasam st...
3,0,ive iphone 8 plus iphone x id say iphone 8 plu...
4,0,iphone x 150 dickhead written
5,1,iphone x still sexy year later
7,1,im using kidney omo ferrari buggati iphone x m...
8,1,iphone x max sale
9,0,someone told thatbut ive never seen ios turned...
10,1,screenshots iphone x look clapped


In [10]:
t=Tokenizer(num_words=50000,lower=True)
t.fit_on_texts(data['text'])
x=t.texts_to_sequences(data['text'])
x=sequence.pad_sequences(x,maxlen=250)
print(x.shape)
print(x)

(904, 250)
[[   0    0    0 ...    2    2  116]
 [   0    0    0 ...  878    1    2]
 [   0    0    0 ...  879  266  880]
 ...
 [   0    0    0 ...    3  260   11]
 [   0    0    0 ...    1    2    8]
 [   0    0    0 ...  513 2335  641]]


In [30]:
# cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)
# data["text"].iplot(kind="bar")

# data['text'].iplot(
#     kind='hist',
#     bins=100,
#     xTitle='review length',
#     linecolor='white',
#     yTitle='count',
#     title='Review Text Length Distribution')




## Spliting of dataset

In [13]:
y=pd.get_dummies(data['class']).values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)
y

(813, 250) (91, 250)
(813, 2) (91, 2)


array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

## Training LSTM model

In [14]:
model=Sequential()
model.add(Embedding(50000,100,input_length=x.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

fit=model.fit(x_train,y_train,epochs=5,batch_size=64,validation_split=0.1)

Epoch 1/5
12/12 [==============================] - 33s 675ms/step - loss: 0.6762 - accuracy: 0.6216 - val_loss: 0.6761 - val_accuracy: 0.5732
Epoch 2/5
12/12 [==============================] - 8s 665ms/step - loss: 0.6421 - accuracy: 0.6249 - val_loss: 0.6592 - val_accuracy: 0.5732
Epoch 3/5
12/12 [==============================] - 7s 619ms/step - loss: 0.6096 - accuracy: 0.6212 - val_loss: 0.6156 - val_accuracy: 0.6341
Epoch 4/5
12/12 [==============================] - 7s 617ms/step - loss: 0.5134 - accuracy: 0.7710 - val_loss: 0.4957 - val_accuracy: 0.8049
Epoch 5/5
12/12 [==============================] - 7s 607ms/step - loss: 0.3274 - accuracy: 0.8985 - val_loss: 0.4678 - val_accuracy: 0.7561


In [15]:
acr=model.evaluate(x_test,y_test)

3/3 [==============================] - 0s 55ms/step - loss: 0.5511 - accuracy: 0.8022


In [16]:
# model=Sequential()
# model.add(Embedding(50000,100,input_length=x.shape[1]))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
# model.add(Dense(2,activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# fit=model.fit(x_train,y_train,epochs=5,batch_size=64,validation_split=0.1)

In [38]:
y_test

array([[0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0,

In [39]:
# cal=t.texts_to_sequences(test)
cal=sequence.pad_sequences(x,maxlen=250)
pred=model.predict_classes(x_test)
pred=np.reshape(pred,(len(pred),1))
pred

array([[1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
    

In [19]:
y=list(y_test)
y2=[]
for i in range(len(y)):
    if y[i][0]==1:
        y2.append(0)
    else:
        y2.append(1)
len(y2)

91

In [20]:
y_true = y2
y_pred = list(pred)
target_names = ['no','yes']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

          no       0.82      0.64      0.72        36
         yes       0.79      0.91      0.85        55

    accuracy                           0.80        91
   macro avg       0.81      0.77      0.78        91
weighted avg       0.80      0.80      0.80        91

